In [1]:
PROJECT_ID = "test-genie-install4"
REGION = "us-central1"
import os
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["PG_HOST"] = ""

In [2]:
import sys
sys.path.append("../../common/src")
sys.path.append("../src")

In [3]:
from google.cloud import artifactregistry_v1
from google.cloud.artifactregistry_v1 import ListTagsRequest

In [27]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=2DhOxQAX0wTfh0WBoHDgYVpXX10b4H&access_type=offline&code_challenge=ffIDQbfSPnV5GShufa6BOKkQBs8lnHgxCEpie7ppRxc&code_challenge_method=S256


Credentials saved to file: [/Users/lramsey/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "test-genie-install4" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [28]:
client = artifactregistry_v1.ArtifactRegistryClient()

In [55]:
project_id = "test-genie-install4"
location = "us"
repository = "default"
package_name = "webscraper"
image_name = "us-docker.pkg.dev/test-genie-install4/default/webscraper:v0.3.2-412-gc9f60f0d-dirty"

In [47]:
parent = client.package_path(project_id, location, repository, package)

In [48]:
parent

'projects/test-genie-install4/locations/us/repositories/default/packages/webscraper'

In [49]:
request = ListTagsRequest(
        parent=parent
)

In [50]:
tags = client.list_tags(request)

In [51]:
tags

ListTagsPager<tags {
  name: "projects/test-genie-install4/locations/us/repositories/default/packages/webscraper/tags/v0.3.2-412-gc9f60f0d-dirty"
  version: "projects/test-genie-install4/locations/us/repositories/default/packages/webscraper/versions/sha256:51d963428ff697497b5287578ebbcdfd2527c4af35619772fbeca664bab28a9d"
}
>

In [64]:
import logging
def get_latest_artifact_registry_image(repository: str, package_name: str, location="us", project_id=None):
  """Get the latest image version from Artifact Registry

  Args:
      repository: Name of the Artifact Registry repository
      package_name: Name of the container package
      location: Registry location (defaults to 'us')
      project_id: GCP project ID (defaults to None)

  Returns:
      Full image path including latest version tag
  """
  try:
    client = artifactregistry_v1.ArtifactRegistryClient()

    parent = client.package_path(project_id, location, repository, package_name)

    # List tags for the package
    request = ListTagsRequest(
        parent=parent,
    )

    # Get all tags and find the latest
    tags = client.list_tags(request)
    latest_tag = None
    latest_version = None

    for tag in tags:
      if tag.version:
        if not latest_version or tag.version > latest_version:
          latest_tag = tag
          latest_version = tag.version

    if latest_tag:
      return latest_tag.name

    raise Exception(f"No versions found for image {package_name}")

  except Exception as e:
    logging.error(f"Error getting latest image version: {str(e)}")
    # Fall back to default image path without version
    return f"{location}-docker.pkg.dev/{project_id}/{repository}/{package_name}"


In [65]:
get_latest_artifact_registry_image(repository, package_name, location, project_id)

'projects/test-genie-install4/locations/us/repositories/default/packages/webscraper/tags/v0.3.2-412-gc9f60f0d-dirty'